In [2]:
#standard libraries to manipulate and slice data
import os
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

In [3]:
#deep learning libraries
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda, Flatten
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
def unet_model(img_height, img_width, channels):

    #Input
    inputs = Input((img_height, img_width, channels))
    inputs = Lambda(lambda x: x / 255)(inputs) #Normalize the pixels by dividing by 255

    #Encoder where we are extracting the features
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    convolution1 = BatchNormalization()(convolution1) #other option is to do batch normalization
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(convolution1)
    pooling1 = MaxPooling2D((2, 2))(convolution1)
      
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    convolution1 = BatchNormalization()(convolution1) #other option is to do batch normalization
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(convolution1)
    pooling1 = MaxPooling2D((2, 2))(convolution1)
    
    convolution2 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(pooling1)
    convolution2 = BatchNormalization()(convolution2)
    convolution2 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(convolution2)
    pooling2 = MaxPooling2D((2, 2))(convolution2)
     
    convolution3 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(pooling2)
    convolution3 = BatchNormalization()(convolution3)
    convolution3 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(convolution3)
    pooling3 = MaxPooling2D((2, 2))(convolution3)
     
    convolution4 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(pooling3)
    convolution4 = BatchNormalization()(convolution4)
    convolution4 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(convolution4)
    pooling4 = MaxPooling2D(pool_size=(2, 2))(convolution4)

    #Bottleneck at the base of the U-net 
    convolution5 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(pooling4)
    convolution5 = BatchNormalization()(convolution5)
    convolution5 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(convolution5)
    
    #Decoder where we are indicating to the model the precise location of the features 
    transconv6 = Conv2DTranspose(128, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution5)
    transconv6 = concatenate([transconv6, convolution4])
    convolution6 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(transconv6)
    convolution6 = BatchNormalization()(convolution6)
    convolution6 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(convolution6)
     
    transconv7 = Conv2DTranspose(64, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution6)
    transconv7 = concatenate([transconv7, convolution3])
    convolution7 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(transconv7)
    convolution7 = BatchNormalization()(convolution7)
    convolution7 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(convolution7)
     
    transconv8 = Conv2DTranspose(32, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution7)
    transconv8 = concatenate([transconv8, convolution2])
    convolution8 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(transconv8)
    convolution8 = BatchNormalization()(convolution8)
    convolution8 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(convolution8)
     
    transconv9 = Conv2DTranspose(16, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution8)
    transconv9 = concatenate([transconv9, convolution1], axis=3)
    convolution9 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(transconv9)
    convolution9 = BatchNormalization()(convolution9)
    convolution9 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(convolution9)
     
    outputs = Conv2D(1, kernel_size=(1, 1), activation='sigmoid')(convolution9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    jaccard = MeanIoU(2)
    
    
    #loss options include: binary_crossentropy, IoU Loss (Jaccard Index), dice loss
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', jaccard])
    
    model.summary()
    
    return model

In [ ]:
model = unet_model(256, 256, 3)

In [ ]:
from skimage import io

In [ ]:
images_path = '../raw_data/Small_dataset/Images'

In [ ]:
target_path = '../raw_data/Small_dataset/Target'

In [ ]:
files = os.listdir(images_path)
target_files = os.listdir(target_path)
target_files

In [ ]:
'string'.endswith('ng')

In [ ]:
images = []
for file in files:
    if file.endswith('Identifier'):
        pass
    else:
        images.append(file)

In [ ]:
data = '/home/areisdorf/code/areisdorf45/solar_panel_segmentation/raw_data/Small_dataset/Images'

In [ ]:
image_results = []

for image in images:
    image = cv.imread(f'{data}/{file}')
    image_results.append(image)
    
image_array = np.array(image_results)
image_array.shape



In [ ]:
targets = []
for file in target_files:
    if file.endswith('Identifier'):
        pass
    else:
        targets.append(file)

In [ ]:
len(targets)

In [ ]:
target_data = '/home/areisdorf/code/areisdorf45/solar_panel_segmentation/raw_data/Small_dataset/Target'

In [ ]:
target_results = []

for file in targets:
    target = io.imread(f'{target_data}/{file}')
    target_results.append(target)
    
target_array = np.array(target_results)
target_array.shape

In [ ]:
y = target_array

In [ ]:
X = image_array

In [ ]:
checkpoint_filepath = '/tmp/checkpoint'
es = EarlyStopping(patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('solar_unet_inital.h5', restore_best_weights=True)

In [ ]:
model.fit(X,y, validation_split=0.2, epochs = 50, batch_size=32, callbacks=[es], verbose=1)

The IoU metric, or Jaccard Index, is similar to the Dice metric and is calculated as the ratio between the overlap of the positive instances between two sets, and their mutual combined values: